# METOD algorithm - Sum of Gaussians

# 1. Import libraries

The following libraries are required in order for the METOD Algorithm to run.

In [ ]:
import numpy as np
from numpy import linalg as LA
import pandas as pd

import metod_testing as mtv3

# 2. Define function and gradient

Weighted sum of Gaussians objectve function:

\begin{equation}
\label{eq:funct1}
f(x_n^{(k)})= -\sum_{p=1}^{P} c_p\exp \Bigg\{ {-\frac{1}{2 \sigma^2}(x_n^{(k)}-x_{0p})^T A_p^T \Sigma_p A_p(x_n^{(k)}-x_{0p})}\Bigg\}\, .
\end{equation}
where $x_n^{(k)}$ ($n=1,...,N$) is a point with $k$ iterations of steepest descent applied, $P$ is the number of Gaussian densities; $A_p$ ($p=1,...,P$) are randomly chosen rotation matrices of size $d\times d$; $\Sigma_p$ ($p=1,...,P$) are diagonal positive definite matrices of size $d\times d$;  $x_{0p}$ ($p=1,...,P$) are random points in $\mathfrak{X}$ (centers of the Gaussian densities); $c_p$ ($p=1,...,P$) are fixed constants.

Note that steepest descent iterations are terminated at the smallest $k=K_n$ such that $\nabla f(x_n^{(k)}) < \delta$. 

The gradient is:
\begin{equation}
\nabla f(x_n^{(k)})= \sum_{p=1}^{P}\frac{c_p} { \sigma^2} \exp \Bigg\{ {-\frac{1}{2 \sigma^2}(x_n^{(k)}-x_{0p})^TA_p^T \Sigma_p A(x_n^{(k)}-x_{0p}} ) \Bigg\} A_p^T \Sigma_p A(x_n^{(k)}-x_{0p})\, .
\end{equation}




In [ ]:
f = mtv3.sog_function
g = mtv3.sog_gradient

# 3. Defining parameters

The following parameters are required in order to derive $A_p$ ($p=1,...,P$) ; $\Sigma_p$ ($p=1,...,P$); $x_{0p}$ ($p=1,...,P$) and $c_p$ ($p=1,...,P$).

We have:

•d: dimension

•P: number of minima

•lambda_1: smallest eigenvalue of $\Sigma_p$ ($p=1,...,P$)

•lambda_2: largest eigenvalue of $\Sigma_p$ ($p=1,...,P$)

•sigma_sq: value for $\sigma^2$

In [ ]:
d = 100
P = 5
lambda_1 = 1
lambda_2 = 10
sigma_sq = 4

In [ ]:
store_x0, matrix_combined, store_c = mtv3.function_parameters_sog(P, d, lambda_1, lambda_2)

We have:

•store_x0: $x_{0p}$ ($p=1,...,P$)

•matrix_combined: $A_p^T \Sigma_p A_p$ ($p=1,...,P$)

•store_c: $c_p$ ($p=1,...,P$)

In [ ]:
args = P, sigma_sq, store_x0, matrix_combined, store_c

In [ ]:
f = mtv3.sog_function
g = mtv3.sog_gradient

# 4. Run METOD Algorithm

Here we run the METOD algorithm. If you would like to change any of the METOD algorithm parameters, please see, 'Solver for Multistart with early termination of  descents (METOD)'. 

In [ ]:
discovered_minima, number_minima, func_vals_of_minima, excessive_no_descents  = mtv3.metod(f, g, args, d)

# 5. Results of the METOD Algorithm

Total number of minima found:

In [ ]:
number_minima

Positions of minima:

In [ ]:
discovered_minima

Function values of minima:

In [ ]:
func_vals_of_minima

Total number of extra descents to an already discovered minima:

In [ ]:
excessive_no_descents

# 6. Save results to csv file (optional)

The below csv files will be saved to the same folder which contains the METOD Algorithm - Minimum of several quadratic forms notebook.

Minima in msqf_discovered_minimas_d_%s_p_%s.csv are saved such that each row represents one discovered minima. The total number of rows will be the same as the value for number_minima.

In [ ]:
np.savetxt('sog_discovered_minimas_d_%s_p_%s.csv' % (d, P), discovered_minima, delimiter=",")

Values in msqf_func_vals_discovered_minimas_d_%s_p_%s are saved such that each row represents the function value of a minima. The total number of rows will be the same as the value for number_minima.

In [ ]:
np.savetxt('sog_func_vals_discovered_minimas_d_%s_p_%s.csv' % (d, P), func_vals_of_minima, delimiter=",")

msqf_summary_table_d_%s_p_%s.csv will contain the total number of minima discovered and the total number of extra descents.

In [ ]:
summary_table = pd.DataFrame({
"Total number of unique minima": [number_minima],
"Extra descents": [excessive_no_descents]})
summary_table.to_csv('sog_summary_table_d_%s_p_%s.csv' % (d, P))

# 7. Test results (optional)

This test can only be used for the sum of Gaussians function.

To check each discovered minima is unique, for each minima $x_l^{(K_l)}$ found where $l=1,...,L$, we do the following:

\begin{equation}
p_l = {\rm argmin}_{1\le p \le P} \|x_l^{(K_l)} - x_{0p}\|
\end{equation}

For $p_l$ found, ensure that:
$$\|x_l^{(K_l)} - x_{0p_l}\| \text{  is small}$$

If all $p_l$ is different for $l=1,...,L$ and also $\|x_l^{(K_l)} - x_{0p_l}\|$ is small for each $p_l$, then all discovered minimas are unique. 

In [ ]:
def calc_minima(point, p, store_x0):
    dist = np.zeros((p))
    for i in range(p):
        dist[i] = LA.norm(point - store_x0[i])
    return np.argmin(dist), np.min(dist)

In [ ]:
"""Store values from calc_minima function""" 
norms_with_minima = np.zeros((number_minima))
pos_list = np.zeros((number_minima))
for j in range(number_minima):
    pos, min_dist = calc_minima(discovered_minima[j], P, store_x0)
    pos_list[j] = pos
    norms_with_minima[j] = min_dist

${\max}_{1\le l \le L}  \|x_l^{(K_l)}-x_{0p_l}\|$ should be small

In [ ]:
np.max(norms_with_minima)

Ensure that the number of unique minima is $L$

In [ ]:
np.unique(pos_list).shape[0] == number_minima